# **Prueba Analistas**

In [1]:
# importar los datos

import pandas as pd
import numpy as np


datos = pd.read_csv('analista_de_operaciones.csv', encoding='latin1', header=0)

print(datos.shape)
print(datos.head())


(105388, 14)
  regional  ciudad codtrabajo       fecha    cuenta      orden  cantidad  \
0   CENTRO  BOGOTA     VENTEC  01/07/2023  15589183  380534979         2   
1   CENTRO  BOGOTA    NROUTER  01/07/2023  15434166  380582904         2   
2   CENTRO  BOGOTA    NROUTER  01/07/2023  15581974  380597816         2   
3   CENTRO  BOGOTA    NROUTER  01/07/2023  15634211  380592700         2   
4   CENTRO  BOGOTA    NROUTER  01/07/2023  15589183  380534979         2   

                     nombre_tecnico      cedula    tipo_trabajo  \
0         JEISON ADAN MOSQUERA RUIZ  1087782391  VENTAS TECNICO   
1  ALEJANDRO MARIO HERNANDEZ PERTUZ  1143432760  VENTAS TECNICO   
2         JORGE ENRIQUE RENZA ALVIZ  1130588089  VENTAS TECNICO   
3      EYSON FERNANDO IJAJI SAMBONI  1107077888  VENTAS TECNICO   
4         JEISON ADAN MOSQUERA RUIZ  1087782391  VENTAS TECNICO   

   precio_trabajo  tipo_movil esquema_trabajo  coordinador  
0         20028.0  MOTO DOBLE             HFC          NaN  
1    

In [2]:
# tipo de datos

type(datos)

pandas.core.frame.DataFrame

In [3]:
# cantidad de datos 

datos.shape

(105388, 14)

In [4]:
# las columnas

datos.columns

Index(['regional', 'ciudad', 'codtrabajo', 'fecha', 'cuenta', 'orden',
       'cantidad', 'nombre_tecnico', 'cedula', 'tipo_trabajo',
       'precio_trabajo', 'tipo_movil', 'esquema_trabajo', 'coordinador'],
      dtype='object')

In [5]:
# Limpiar los nombres de las columnas de posibles espacios en blanco
datos.columns = datos.columns.str.strip()

# Imprimir los nombres de las columnas nuevamente para verificar
print(datos.columns)


Index(['regional', 'ciudad', 'codtrabajo', 'fecha', 'cuenta', 'orden',
       'cantidad', 'nombre_tecnico', 'cedula', 'tipo_trabajo',
       'precio_trabajo', 'tipo_movil', 'esquema_trabajo', 'coordinador'],
      dtype='object')


## **Promedio Servicio**

In [6]:
# promedio de trabajo

promedio_precio = round(datos['precio_trabajo'].mean(), 2)
print(promedio_precio)


52897.61


# **Dataframe Tecnico**

In [7]:
# Agrupar por el nombre del técnico
df_tecnico = datos.groupby('nombre_tecnico').agg(
    Numero_Servicios=('precio_trabajo', 'count'),
    Valor_Total=('precio_trabajo', 'sum')
).reset_index()

# Agregar la columna de clasificación según el Valor Total
def clasificar(valor_total):
    if valor_total <= 5000000:
        return "MUY MALO"
    elif valor_total <= 10000000:
        return "MALO"
    elif valor_total <= 15000000:
        return "REGULAR"
    elif valor_total <= 20000000:
        return "BUENO"
    else:
        return "EXCELENTE"

df_tecnico['Clasificación'] = df_tecnico['Valor_Total'].apply(clasificar)

# Ver el resultado
print(df_tecnico)

                        nombre_tecnico  Numero_Servicios  Valor_Total  \
0                     ABISAI GIL GOMEZ               161    8179974.0   
1          ACERO LEON EDUARD ALEXANDER               116    4196428.0   
2           ACOSTA BOSSA JORGE ELIECER               143    6857306.0   
3      ADRIAN ALONSO RINCON AVELLANEDA               191   11548276.0   
4    AGUDELO RODRIGUEZ FABIAN ESNEIDER                16     667627.0   
..                                 ...               ...          ...   
698  YILBER NICOLAS BUITRAGO HERNANDEZ                61    2831577.0   
699        YONATHA STID MOGOLLON ANAYA                13     445286.0   
700  YONATHAN ARLEY CASTILLO GUTIERREZ                92    9167403.0   
701         YORFAN SMITH GUIZA VELASCO               116    8527798.0   
702      YORMAN ENRIQUE GONZALEZ PABON               170    6093439.0   

    Clasificación  
0            MALO  
1        MUY MALO  
2            MALO  
3         REGULAR  
4        MUY MALO  
.. 

In [8]:
print(df_tecnico.columns)


Index(['nombre_tecnico', 'Numero_Servicios', 'Valor_Total', 'Clasificación'], dtype='object')


## **Exportar la tabla**

In [9]:
# Exportar la tabla

df_tecnico.to_csv('df_tecnico.csv', index=False)

# **Porcentajes y Servicios**

In [11]:
# Agrupar y calcular totales
df_agrupado = datos.groupby(['tipo_movil', 'esquema_trabajo']).agg(
    Valor_Total=('precio_trabajo', 'sum')
).reset_index()

# Calcular el total global para porcentaje
total_global = datos['precio_trabajo'].sum()

# Calcular porcentaje
df_agrupado['Porcentaje'] = (df_agrupado['Valor_Total'] / total_global) * 100

# Redondear
df_agrupado['Valor_Total'] = df_agrupado['Valor_Total'].round(2)
df_agrupado['Porcentaje'] = df_agrupado['Porcentaje'].round(2)

# Pivotear
tabla_pivot = df_agrupado.pivot(index='tipo_movil', columns='esquema_trabajo', values=['Valor_Total', 'Porcentaje'])

# Rellenar NaN con 0
tabla_pivot = tabla_pivot.fillna(0)

# Formatear los valores
tabla_pivot_formateada = tabla_pivot.copy()
tabla_pivot_formateada['Valor_Total'] = tabla_pivot['Valor_Total'].map(lambda x: f"{x:,.2f}")
tabla_pivot_formateada['Porcentaje'] = tabla_pivot['Porcentaje'].map(lambda x: f"{x:.2f}%")

# Unir los valores formateados
tabla_final = pd.concat([tabla_pivot_formateada['Valor_Total'], tabla_pivot_formateada['Porcentaje']], axis=1)

# Mostrar tabla final
print(tabla_final)

esquema_trabajo            FTTH               HFC    FTTH     HFC
tipo_movil                                                       
CAMIONETA        235,443,776.00     70,477,795.00   4.22%   1.26%
MOTO DOBLE       955,828,294.00  2,354,511,815.00  17.15%  42.24%
MOTO SENCILLA    111,927,593.00    779,504,820.00   2.01%  13.98%
MOTO SENCILLA        118,406.00         77,363.00   0.00%   0.00%
MOVIL ARREGLOS   270,721,047.00    780,957,647.00   4.86%  14.01%
MOVIL PYMES                0.00     15,205,180.00   0.00%   0.27%


In [15]:
# segundo excel

tabla_final.to_excel('tabla_final.xlsx', index=True)

# **Codtrabajo y Cantidad**


In [16]:
# Agrupar por 'codtrabajo' y sumar la cantidad de trabajos
df_codtrabajo_cantidad = datos.groupby('codtrabajo').agg({'cantidad': 'sum', 'precio_trabajo': 'sum'}).reset_index()

# Mostrar el DataFrame resultante
print(df_codtrabajo_cantidad)

    codtrabajo  cantidad  precio_trabajo
0      ARTOC95       101       3850827.0
1      ARTOC96      8916     349293216.0
2      ARTOCNR      1865      65260080.0
3    FTARTOC95        10        774040.0
4    FTARTOC96      1819     144674165.0
..         ...       ...             ...
103  PYARTOC95         2        128496.0
104  PYARTOC96       288      19012608.0
105  PYARTOCNR        58       3418694.0
106   REVISITA      2262             0.0
107     VENTEC      2433      24364062.0

[108 rows x 3 columns]


In [17]:
# tercer excel

df_codtrabajo_cantidad.to_csv('df_codtrabajo_cantidad.csv', index=False)

# **Rellenar la columna de Coordinador**

In [18]:
referencia_coordinadores = pd.DataFrame({
    'ciudad': ['BOGOTA', 'BOGOTA', 'BUCARAMANGA', 'BUCARAMANGA', 'YOPAL', 'YOPAL', 'VILLAVICENCIO', 'VILLAVICENCIO'],
    'esquema_trabajo': ['HFC', 'FTTH', 'HFC', 'FTTH', 'HFC', 'FTTH', 'HFC', 'FTTH'],
    'coordinador': ['GIOVANNI V', 'JHON T', 'DIEGO S', 'EDINSON Q', 'HUBER M', 'JHON G', 'SERGIO G', 'ANDRES G']
})

In [19]:
# Asegurarse de que los textos estén en mayúscula para que el merge funcione correctamente
datos['ciudad'] = datos['ciudad'].str.upper()
datos['esquema_trabajo'] = datos['esquema_trabajo'].str.upper()


In [20]:
# Llenar la columna 'coordinador' uniendo por ciudad y esquema de trabajo
datos = datos.drop(columns=['coordinador'], errors='ignore')  # eliminar si ya existe vacía
datos = datos.merge(referencia_coordinadores, on=['ciudad', 'esquema_trabajo'], how='left')


In [21]:
datos

,regional,ciudad,codtrabajo,fecha,cuenta,orden,cantidad,nombre_tecnico,cedula,tipo_trabajo,precio_trabajo,tipo_movil,esquema_trabajo,coordinador
0,CENTRO,BOGOTA,VENTEC,01/07/2023,15589183,380534979,2,JEISON ADAN MOSQUERA RUIZ,1087782391,VENTAS TECNICO,20028.0,MOTO DOBLE,HFC,GIOVANNI V
1,CENTRO,BOGOTA,NROUTER,01/07/2023,15434166,380582904,2,ALEJANDRO MARIO HERNANDEZ PERTUZ,1143432760,VENTAS TECNICO,33090.0,MOTO DOBLE,HFC,GIOVANNI V
2,CENTRO,BOGOTA,NROUTER,01/07/2023,15581974,380597816,2,JORGE ENRIQUE RENZA ALVIZ,1130588089,VENTAS TECNICO,33090.0,MOTO DOBLE,HFC,GIOVANNI V
3,CENTRO,BOGOTA,NROUTER,01/07/2023,15634211,380592700,2,EYSON FERNANDO IJAJI SAMBONI,1107077888,VENTAS TECNICO,33090.0,MOTO DOBLE,HFC,GIOVANNI V
4,CENTRO,BOGOTA,NROUTER,01/07/2023,15589183,380534979,2,JEISON ADAN MOSQUERA RUIZ,1087782391,VENTAS TECNICO,33090.0,MOTO DOBLE,HFC,GIOVANNI V
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105383,ORIENTE,VILLAVICENCIO,ARTOC96,31/07/2023,24820037,8798206,1,CURREA HERRERA HUGO ALFONSO,1121871475,MANTENIMIENTOS,39176.0,MOVIL ARREGLOS,FTTH,ANDRES G
105384,CENTRO,BOGOTA,NDESCO,31/07/2023,16170421,381960297,1,CAMILLO SPECIA CARVAJAL,1126625677,DESCONEXIONES,8746.0,MOTO SENCILLA,HFC,GIOVANNI V
105385,CENTRO,BOGOTA,NFTICTNC,31/07/2023,19680087,382136645,1,DEIMER ALONSO ARTUZ JARAMILLO,1073990828,INSTALACIONES FTTH,178192.0,MOTO DOBLE,FTTH,JHON T
105386,CENTRO,BOGOTA,NITVCA,31/07/2023,25973369,382203547,1,CARLOS EDUARDO DAJOME VALENCIA,1023952580,INSTALACIONES ALTO VALOR,74622.0,MOTO DOBLE,HFC,GIOVANNI V


In [23]:
#  excel final

datos.to_csv('datos.csv', index=False)